### 2D CNN Model

In [1]:
from __future__ import print_function
from hyperas import optim
from hyperas.distributions import uniform
from hyperopt import Trials, STATUS_OK, tpe
# https://github.com/maxpumperla/hyperas

C:\Users\Nah\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

import h5py

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
def Data_Prep():
    nb_classes = 1
    
    data = pd.read_csv('./2D_120.csv')

    target = data.AGE.copy(deep=True)
    data = data.drop(['Unnamed: 0','AGE'],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
    data,target,test_size=0.2,random_state=42)

    mm = MinMaxScaler()
    X_train = mm.fit_transform(X_train)
    X_test  = mm.transform(X_test)

    X_train = X_train.reshape(X_train.shape[0], 256, 150, 1)
    X_test  = X_test.reshape(X_test.shape[0], 256, 150, 1)
    return X_train, X_test, y_train, y_test

In [4]:
def model(X_train, Y_train, X_test, Y_test):

    nb_epoch = 15

    model = Sequential()
    model.add(Conv2D({{choice([30, 45, 60])}}, 
                     (5,5), padding='same',
                     activation='relu',
                    input_shape=X_train.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Conv2D({{choice([30, 45, 60])}}, 
                     (4,4), padding='same', 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Flatten())
    model.add(Dense({{choice([512, 256])}},activation='relu'))
    
    model.add(Dense(1,activation='relu'))

    model.compile(loss='mean_absolute_error',
                  optimizer=Adam(), 
                  metrics=['mean_absolute_error'])

    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, mode='auto')
    
    model.fit(X_train, y_train,
            batch_size=128,
            epochs=nb_epoch,
            callbacks=[early_stop], 
            validation_data=(X_test, y_test))

    score, acc = model.evaluate(X_test, y_test, verbose=1)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
#X_train, X_test, y_train, y_test = Data_Prep()

In [ ]:
best_run, best_model, space = optim.minimize(model=model,
                        data=Data_Prep, algo=tpe.suggest,
                        max_evals=3,  trials=Trials(),
                        notebook_name='2D_CNN_Model_Hyperas')

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.models import Model, Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.layers.convolutional import MaxPooling2D
except:
    pass

try:
    from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import h5py
except:
    pass

try:
    import pandas as pd
except:
    pass

tr

Epoch 15/15
104/104 [==============================] - 2s 23ms/step
Train on 412 samples, validate on 104 samples
Epoch 1/15
412/412 [==============================] - 40s 96ms/step - loss: 2382.4528 - mean_squared_error: 2382.4528 - val_loss: 1599.9119 - val_mean_squared_error: 1599.9119
Epoch 2/15
412/412 [==============================] - 39s 95ms/step - loss: 1471.4769 - mean_squared_error: 1471.4769 - val_loss: 1462.1970 - val_mean_squared_error: 1462.1970
Epoch 3/15
412/412 [==============================] - 39s 96ms/step - loss: 1092.8155 - mean_squared_error: 1092.8155 - val_loss: 985.0215 - val_mean_squared_error: 985.0215
Epoch 4/15
412/412 [==============================] - 40s 97ms/step - loss: 1051.4707 - mean_squared_error: 1051.4707 - val_loss: 907.3532 - val_mean_squared_error: 907.3532
Epoch 5/15
412/412 [==============================] - 40s 97ms/step - loss: 717.5224 - mean_squared_error: 717.5224 - val_loss: 734.3369 - val_mean_squared_error: 734.3369
Epoch 6/15
412

In [ ]:
sns.regplot(best_model.predict(X_test), y_test)
#sns.plt.show()